The directory structure should look like this.

model/
    fisheries_model.ipynb
    vgg16bn.py
    vgg16.py
    utils.py
    data/
        fisheries/
            train/
            test/
            valid/
            results/
            models/

In [1]:
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = LESSON_HOME_DIR+'/data/fisheries'
path = DATA_HOME_DIR + '/' 

In [3]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: zero-size array to reduction operation maximum which has no identity

In [3]:
test = get_data(path+'test', (360,640))

Found 1000 images belonging to 1 classes.


In [4]:
from vgg16bn import Vgg16BN

In [5]:
vgg640 = Vgg16BN((360, 640)).model
vgg640.pop()
vgg640.input_shape, vgg640.output_shape
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [6]:
conv_test_feat = vgg640.predict(test, batch_size=32, verbose=1)

1000/1000 [==============================] - 118s   


In [7]:
save_array(path+'results/conv_test_640.dat', conv_test_feat)

In [6]:
conv_test_feat = load_array(path+'results/conv_test_640.dat')

IOError: [Errno 2] No such file or directory: '/home/ubuntu/nbs/test/data/fisheries/results/conv_test_640.dat/meta/sizes'

In [7]:
conv_layers,_ = split_at(vgg640, Convolution2D)

In [8]:
nf=128; p=0.

In [9]:
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(8,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

In [10]:
lrg_model = Sequential(get_lrg_layers())

th


In [11]:
lrg_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
lrg_model.load_weights(path+'models/lrg1_nmp.h5')

IOError: Unable to open file (Unable to open file: name = '/home/ubuntu/nbs/test/data/fisheries/models/lrg1_nmp.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [15]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/7, mx)

In [16]:
batch_size = 64

In [17]:
preds = lrg_model.predict(conv_test_feat, batch_size=batch_size)

In [18]:
subm = do_clip(preds,0.99)

In [19]:
subm_name = path+'results/subm_test.gz'

In [24]:
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', [a[8:] for a in test_filenames])
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_06237.jpg,0.553382,0.433272,0.006673,0.001429,0.004004,0.001429,0.001429,0.001429
1,img_06893.jpg,0.363925,0.001429,0.006606,0.008224,0.030795,0.385230,0.068290,0.136388
2,img_02082.jpg,0.074978,0.001429,0.001429,0.001429,0.695943,0.001429,0.221384,0.003620
3,img_06261.jpg,0.522467,0.024758,0.001429,0.002223,0.017248,0.023436,0.010176,0.398910
4,img_03628.jpg,0.990000,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429


In [25]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [26]:
FileLink(subm_name)

/home/ubuntu/nbs/data/fisheries/results/subm_test.gz